# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계3 : 응급상황 연계(추천)**

## **0.미션**

단계 3에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### 미션4 : 응급실 추천
* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기


## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/project6/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine
import requests
import json

# 더 필요한 라이브러리 추가 -------------




### (3) 데이터 로딩
* 단계1에서 수집한 응급실 정보를 불러와서 데이터프레임으로 저장합니다.

In [ ]:
data = pd.read_csv(path + "응급실정보.csv")
data

,병원이름,주소,응급의료기관 종류,전화번호,응급실번호,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491
...,...,...,...,...,...,...,...
522,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",지역응급의료센터,031-467-9717,031-467-9119,37.393404,126.924477
523,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",지역응급의료센터,031-389-3000,031-389-3119,37.358645,126.947360
524,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),응급실운영신고기관,051-709-3000,051-709-3119,35.185413,129.121459
525,흑룡의원,인천광역시 옹진군 백령면 백령로 831,응급실운영신고기관,032-837-6873,032-837-3153,37.959524,124.665499


## **2. 응급실 추천**


### (1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
- 세부사항
    * 하버사인 함수를 활용


#### 1) 하버사인 함수 사용 연습
* 임의의 두 좌표간 거리 계산
    * 응급실 데이터프레임을 열어서
    * 응급실 두 곳의 좌표를 확인하고
    * 두 지점의 거리를 계산해 봅시다
* 사용법 : haversine((위도1, 경도1), (위도2, 경도2), unit='km')


In [ ]:

lat1 = data.loc[0, '위도']
long1 = data.loc[0, '경도']
lat2 = data.loc[1, '위도']
long2 = data.loc[1, '경도']
leng = haversine((lat1, long1), (lat2, long2), unit='km')
print(leng)

35.672584868505005


#### 2) 가장 가까운 응급실 3곳 추천하기1
* 세부사항
    * 입력된 좌표와 전체 응급실과 거리를 계산한 후
    * 가장 가까운 거리의 응급실 3 곳을 반환합니다.
* 이를 함수로 생성하고 테스트 해 봅시다.

In [ ]:
def recommend(data, x, y):
    dist = []
    for lat, long in zip(data['위도'], data['경도']):
        dist.append(haversine((lat, long), (x, y), unit='km'))
    data['dist'] = dist
    return data.sort_values(by='dist').head(3)

In [ ]:
x = 37.4606234233867
y = 126.81453997894835
result = recommend(data, x, y)
result

,병원이름,주소,응급의료기관 종류,전화번호,응급실번호,위도,경도,dist
304,의료법인록향의료재단신천연합병원,"경기도 시흥시 복지로 57, 복지로 61, 2층 (대야동)",지역응급의료기관,031-310-6300,031-310-6661,37.444546,126.789384,2.850798
154,부천세종병원,"경기도 부천시 소사구 호현로489번길 28 (소사본동, 세종병원)",지역응급의료센터,1599-6677,032-340-1119,37.481005,126.791167,3.064438
9,가톨릭대학교부천성모병원,"경기도 부천시 원미구 소사로 327, 가톨릭대학교 부천성모병원 (소사동)",지역응급의료센터,032-1577-0675,032-340-2100,37.487448,126.792435,3.564002


In [ ]:
x = 37.75085845594263
y = 128.912838107585
result = recommend(data, x, y)
result

,병원이름,주소,응급의료기관 종류,전화번호,응급실번호,위도,경도,dist
390,의산의료재단강릉고려병원,강원특별자치도 강릉시 옥가로 30 (옥천동),응급실운영신고기관,033-642-1988,033-647-7582,37.759101,128.899825,1.465918
27,강원특별자치도강릉의료원,강원특별자치도 강릉시 경강로 2007 (남문동),지역응급의료기관,033-610-1200,033-610-1234,37.749310,128.888796,2.120767
286,의료법인강릉동인병원,강원특별자치도 강릉시 강릉대로419번길 42 (포남동),지역응급의료센터,033-650-6113,033-650-6105,37.774326,128.907142,2.657063


#### 3) 가장 가까운 응급실 3곳 추천하기2
* 문제점 : 입력 받은 좌표와 응급실 전체와의 거리를 모두 계산하는 것은 비효율 적입니다.
* 해결 방안 : 그래서 입력 받은 좌표를 기준으로 일정 범위 내에 해당되는 응급실에 대해서 거리를 계산하고 추천하도록 기존 함수를 수정 합니다.
* hint :
    * 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고
    * 응급실 정보에서 해당 구간을 먼저 조회한 후
    * 거리 계산

In [ ]:
def recommend_nn(data, x, y):
    dist = []
    neighbor = data.loc[
    (abs(data['위도'] - x) <= 0.3) &
    (abs(data['경도'] - y) <= 0.3)].copy()

    if len(neighbor) <= 3:
        neighbor = data.loc[
        (abs(data['위도'] - x) <= 0.5) &
        (abs(data['경도'] - y) <= 0.5)].copy()
    for lat, long in zip(neighbor['위도'], neighbor['경도']):
        dist.append(haversine((lat, long), (x, y), unit='km'))
    neighbor['dist'] = dist
    return neighbor.sort_values(by='dist').head(3)

In [ ]:
x = 37.75085845594263
y = 128.912838107585

result = recommend_nn(data, x, y)
result

,병원이름,주소,응급의료기관 종류,전화번호,응급실번호,위도,경도,dist
390,의산의료재단강릉고려병원,강원특별자치도 강릉시 옥가로 30 (옥천동),응급실운영신고기관,033-642-1988,033-647-7582,37.759101,128.899825,1.465918
27,강원특별자치도강릉의료원,강원특별자치도 강릉시 경강로 2007 (남문동),지역응급의료기관,033-610-1200,033-610-1234,37.749310,128.888796,2.120767
286,의료법인강릉동인병원,강원특별자치도 강릉시 강릉대로419번길 42 (포남동),지역응급의료센터,033-650-6113,033-650-6105,37.774326,128.907142,2.657063


In [ ]:
x = 37.4606234233867
y = 126.81453997894835

result = recommend_nn(data, x, y)
result

,병원이름,주소,응급의료기관 종류,전화번호,응급실번호,위도,경도,dist
304,의료법인록향의료재단신천연합병원,"경기도 시흥시 복지로 57, 복지로 61, 2층 (대야동)",지역응급의료기관,031-310-6300,031-310-6661,37.444546,126.789384,2.850798
154,부천세종병원,"경기도 부천시 소사구 호현로489번길 28 (소사본동, 세종병원)",지역응급의료센터,1599-6677,032-340-1119,37.481005,126.791167,3.064438
9,가톨릭대학교부천성모병원,"경기도 부천시 원미구 소사로 327, 가톨릭대학교 부천성모병원 (소사동)",지역응급의료센터,032-1577-0675,032-340-2100,37.487448,126.792435,3.564002


### (2) [조 과제]고도화 : naver 지도 api 사용

* 이 부분은 조별 과제로 수행하게 됩니다.(개인과제 아님!)

* 세부사항
    * 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수를 생성하시오.
    * 함수 내용
        * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
        * 출력 : 도로거리(km)
    
    * 네이버 Maps API 활용
        * 사용할 API : Direction 5
        * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
        * 가이드를 활용해서 url, header, params를 구성합니다.
        * params의 옵션은 'trafast' (실시간 빠른 길 옵션)을 선택하시오.

#### 1) maps 클라이언트ID, 키 로딩

In [ ]:
c_id = ''
c_key = '' # 개인 키, 클라이언트 입력

#### 2) 함수 생성

In [ ]:
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers=headers, params=params)
    response = response.json()
    dist = response['route']['trafast'][0]['summary']['distance']  # m(미터)
    duration = response['route']['trafast'][0]['summary']['duration'] # 도착시간(ms)
    return dist, duration

In [ ]:
x = 37.4606234233867
y = 126.81453997894835

result = recommend_nn(data, x, y)
result.reset_index(drop=True, inplace=True)
dest_x = result.loc[0, '위도']
dest_y = result.loc[0, '경도']

* 테스트

In [ ]:
dist, dur = get_dist(x, y, dest_x, dest_y, c_id, c_key)
dist , dur

(3877, 856453)

#### 3) 응급실 추천
* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

In [ ]:
# 도착시간 형식 변경 (ms -> 시 분 초 형식)
def convert_milliseconds(ms):
    hours = ms // 3600000
    ms %= 3600000
    minutes = ms // 60000
    ms %= 60000
    seconds = ms // 1000
    milliseconds = ms % 1000

    return f"{hours}시간 {minutes}분 {seconds}초 {milliseconds}밀리초"

def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):  # 위에 작성된 함수와 동일 -> api로 거리 계산 함수
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers=headers, params=params)
    response = response.json()
    dist = response['route']['trafast'][0]['summary']['distance']  # m(미터)
    duration = response['route']['trafast'][0]['summary']['duration'] # 도착시간
    return dist, duration


In [ ]:
def recommend_api(data, distrange, x, y):
    dist = []
    duration = []
    neighbor = data.loc[
    (abs(data['위도'] - x) <= distrange) &
    (abs(data['경도'] - y) <= distrange)].copy()

    if(len(neighbor)<=3):
      neighbor = data.loc[
      (abs(data['위도'] - x) <= distrange + 0.1) &
      (abs(data['경도'] - y) <= distrange + 0.1)].copy()

    for lat, long in zip(neighbor['위도'], neighbor['경도']):
        di, du = get_dist(x, y, lat, long, c_id, c_key)
        dist.append(di)
        duration.append(du)

    neighbor['dist'] = dist
    neighbor['duration'] = duration
    return neighbor.sort_values(by='dist').head(3), neighbor.sort_values(by='duration').head(3)

In [ ]:
# 테스트 위도 경도 -> 스타필드 성남 근처였던걸로 기억합니당 ^^;
x = 37.4606234233867
y = 126.81453997894835

resultdist, resultdur = recommend_api(data, x, y)

In [ ]:
resultdist

,병원이름,주소,응급의료기관 종류,전화번호,응급실번호,위도,경도,dist,duration
304,의료법인록향의료재단신천연합병원,"경기도 시흥시 복지로 57, 복지로 61, 2층 (대야동)",지역응급의료기관,031-310-6300,031-310-6661,37.444546,126.789384,3877,800040
154,부천세종병원,"경기도 부천시 소사구 호현로489번길 28 (소사본동, 세종병원)",지역응급의료센터,1599-6677,032-340-1119,37.481005,126.791167,4021,1205034
9,가톨릭대학교부천성모병원,"경기도 부천시 원미구 소사로 327, 가톨릭대학교 부천성모병원 (소사동)",지역응급의료센터,032-1577-0675,032-340-2100,37.487448,126.792435,4549,1281978


In [ ]:
resultdur

,병원이름,주소,응급의료기관 종류,전화번호,응급실번호,위도,경도,dist,duration
304,의료법인록향의료재단신천연합병원,"경기도 시흥시 복지로 57, 복지로 61, 2층 (대야동)",지역응급의료기관,031-310-6300,031-310-6661,37.444546,126.789384,3877,800040
154,부천세종병원,"경기도 부천시 소사구 호현로489번길 28 (소사본동, 세종병원)",지역응급의료센터,1599-6677,032-340-1119,37.481005,126.791167,4021,1205034
526,희명병원,서울특별시 금천구 시흥대로 244 (시흥동),지역응급의료기관,02-804-0002,02-809-0122,37.455671,126.900563,9868,1208065


## **Mission Complete!**

수고 많았습니다!